In [105]:
import torch
from kan import *
from my_functions import calculate_signature, calculate_signature_over_all_surface, format_signature
import plotly.graph_objects as go


In [106]:
mesh_path = "data/bunny_curvs.ply"
sampled_index = 30000
dataset = create_dataset_from_mesh(mesh_path=mesh_path,sampled_index=sampled_index, show_sample=True, ball_radius=0.06, down_sampling_ratio=0.8)
x = torch.tensor([[0.0], [0.0]], requires_grad=True).T
dataset['train_input'].shape, dataset['train_label'].shape

(torch.Size([149, 2]), torch.Size([149, 1]))

In [107]:
experiment_num = 10
signatures = torch.zeros(experiment_num, 6)
signatures2 = torch.zeros(experiment_num, 6)
dataset_sampled_points = []
grid_range = 1.0
ball_radius = 0.07
lr = 0.1
for i in range(experiment_num):
    dataset = create_dataset_from_mesh(mesh_path=mesh_path,sampled_index=sampled_index, show_sample=False, ball_radius=ball_radius, seed=i, down_sampling_ratio=0.8)

    model = KAN(width=[2,5,1], grid=3, k=5, grid_eps=1.0, seed=0, grid_range=[-grid_range,grid_range], noise_scale=0.1, noise_scale_base=1.0, base_fun=torch.nn.SiLU(), learn_rotation_mat=False)
    model.train(dataset, opt="LBFGS", steps=20,lr=lr, lamb=100.0, lamb_l1=1.0, lamb_entropy=1.0, lamb_coef=0.0, lamb_coefdiff=0.0, sglr_avoid=True)
    model = model.prune(threshold=3e-3, mode='manual', active_neurons_id=[[0,1],[0]])
    model.train(dataset, opt="LBFGS", steps=50,lr=lr, sglr_avoid=True)
     # manual mode
    # fix all
    model.fix_symbolic(0,0,0,'x^4')
    model.fix_symbolic(0,1,0,'x^3')
    model.fix_symbolic(1,0,0,'x^2')

    # model.fix_symbolic(1,1,0, 'x')
    # model.fix_symbolic(0,0,1,'x^5')
    # model.fix_symbolic(0,1,1,'x')
    # model.fix_symbolic(1,1,0,'x^5')
    # model.fix_symbolic(1,1,0,'1')
    # model.fix_symbolic(1,1,0,'x')
    # model.fix_symbolic(2,0,0,'1')

    # lib = ['x','x^2', 'x^3', 'x^4', 'sqrt']
    # model.auto_symbolic(lib=lib)
    model.learn_rotation_mat = False
    model.train(dataset, opt="LBFGS", steps=50, lr=lr,sglr_avoid=True, lamb_coef=1.0, lamb_coefdiff=1.0, loss_fn=gaussian_weighted_mse)

    print(model.symbolic_formula()[0][0])
    signature = calculate_signature(model, x) # only calc sig in the origin
    # signature2 = calculate_signature_over_all_surface(model, torch.tensor(dataset['train_input'], requires_grad=True)) # calc sig over all surface and take the mean
    print("exp ", i, "signature ", signature)
    signatures[i] = signature
    # signatures2[i] = signature2
    curr_sampled_points = torch.concatenate((dataset['test_input'], model(dataset['test_input'])), dim=1)
    dataset_sampled_points.append(curr_sampled_points)


train loss: 3.82e-02 | test loss: 3.70e-02 | reg: 2.70e+00 : 100%|██| 20/20 [00:02<00:00,  7.52it/s]
train loss: 9.72e-04 | test loss: 4.76e-03 | reg: 1.65e+00 : 100%|██| 50/50 [00:03<00:00, 14.45it/s]


Best value at boundary.
r2 is 0.5971109867095947
r2 is not very high, please double check if you are choosing the correct symbolic function.
r2 is 0.7317348718643188
r2 is not very high, please double check if you are choosing the correct symbolic function.
Best value at boundary.
r2 is 0.9397726655006409


train loss: 2.62e-03 | test loss: 3.91e-03 | reg: 2.94e+00 : 100%|██| 50/50 [00:00<00:00, 55.23it/s]
/Users/gal.yona/Documents/MSc/Thesis/ImplicitNeuralSurfaces/forked_pykan/pykan/kan/KAN.py:344: UserWarning:

std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/ReduceOps.cpp:1760.)



38810174.35*(-x_1 - 0.01)**8 - 0.03
exp  0 signature  tensor([[  -2.5234,    2.1833,  356.9958,    4.7638, -341.5815,  -41.9627]],
       grad_fn=<StackBackward0>)


train loss: 3.67e-02 | test loss: 3.83e-02 | reg: 2.55e+00 : 100%|██| 20/20 [00:02<00:00,  8.69it/s]
train loss: 1.06e-03 | test loss: 5.66e-03 | reg: 1.75e+00 : 100%|██| 50/50 [00:02<00:00, 18.48it/s]


...

train loss: 2.90e-02 | test loss: 2.93e-02 | reg: 2.29e+00 : 100%|██| 20/20 [00:02<00:00,  8.53it/s]
train loss: 9.31e-04 | test loss: 5.88e-03 | reg: 1.75e+00 : 100%|██| 50/50 [00:02<00:00, 17.31it/s]


Best value at boundary.
r2 is 0.610982358455658
r2 is not very high, please double check if you are choosing the correct symbolic function.
r2 is 0.6316137909889221
r2 is not very high, please double check if you are choosing the correct symbolic function.
Best value at boundary.
r2 is 0.9155071377754211


train loss: 2.52e-03 | test loss: 4.73e-03 | reg: 3.20e+00 : 100%|██| 50/50 [00:00<00:00, 51.87it/s]


41897234.11*(-x_1 - 0.01)**8 - 0.03
exp  9 signature  tensor([[  -2.2799,    1.1058,  341.1504,    2.1297, -175.5862,  -17.2102]],
       grad_fn=<StackBackward0>)


/Users/gal.yona/Documents/MSc/Thesis/ImplicitNeuralSurfaces/forked_pykan/pykan/kan/KAN.py:344: UserWarning:

std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/ReduceOps.cpp:1760.)



In [108]:
# plot the sampled points in the same plot but different colors
fig = go.Figure()
for i in range(len(dataset_sampled_points)):
    curr_sampled_points = dataset_sampled_points[i].detach()
    fig.add_trace(go.Scatter3d(x=curr_sampled_points[:,0], y=curr_sampled_points[:,1], z=curr_sampled_points[:,2], mode='markers', marker=dict(size=3), name="exp "+str(i)))
fig.show()

In [109]:
signatures

tensor([[  -2.5234,    2.1833,  356.9958,    4.7638, -341.5815,  -41.9627],
        [  -2.5323,    2.2540,  350.5453,    4.5372, -346.2897,  -39.3455],
        [  -2.5441,    2.3328,  356.5858,    5.0860, -363.8193,  -44.8737],
        [  -2.2138,    1.4416,  328.3057,    2.8818, -232.8502,  -21.7091],
        [  -2.3361,    2.1650,  375.0593,    5.2776, -391.4387,  -42.8116],
        [  -2.8419,    2.5338,  407.0122,    5.1820, -396.8584,  -52.5931],
        [  -2.4083,    1.8468,  344.6872,    4.3046, -289.9984,  -36.4636],
        [  -2.3557,    2.4768,  381.0054,    5.7766, -459.4767,  -46.4667],
        [  -2.5538,    1.8628,  365.7951,    3.6706, -289.4641,  -33.1539],
        [  -2.2799,    1.1058,  341.1504,    2.1297, -175.5862,  -17.2102]],
       grad_fn=<CopySlices>)

In [110]:
 #discard all nan and 0
signatures = signatures[~torch.isnan(signatures).any(dim=1)]
signatures = signatures[~(signatures == 0).all(dim=1)]

In [111]:
mean_signature = torch.mean(signatures, dim=0)
std_signature = torch.std(signatures, dim=0)
print("mean signature ", mean_signature)
print("std signature ", std_signature)



mean signature  tensor([  -2.4589,    2.0203,  360.7142,    4.3610, -328.7363,  -37.6590],
       grad_fn=<MeanBackward1>)
std signature  tensor([ 0.1804,  0.4592, 22.6349,  1.1491, 83.7067, 11.0295],
       grad_fn=<StdBackward0>)


In [112]:
#discard all nan and 0
# signatures2 = signatures2[~torch.isnan(signatures2).any(dim=1)]
# signatures2 = signatures2[~(signatures == 0).all(dim=1)]
# mean_signature2 = torch.mean(signatures2, dim=0)
# std_signature2 = torch.std(signatures2, dim=0)
# print("mean signature ", mean_signature2)
# print("std signature ", std_signature2)


In [113]:
# remove outliers that are 2 stds away from the mean
outliers = torch.abs(signatures - mean_signature) > 2*std_signature
signatures_without_outliers = signatures[~outliers.any(dim=1)]
mean_signature = torch.mean(signatures_without_outliers, dim=0)
std_signature = torch.std(signatures_without_outliers, dim=0)
print("mean signature ", mean_signature)
print("std signature ", std_signature)
print(signatures_without_outliers.shape)
print(signatures_without_outliers)



mean signature  tensor([  -2.4164,    1.9632,  355.5700,    4.2698, -321.1672,  -35.9997],
       grad_fn=<MeanBackward1>)
std signature  tensor([ 0.1275,  0.4479, 16.6934,  1.1797, 85.0772, 10.2898],
       grad_fn=<StdBackward0>)
torch.Size([9, 6])
tensor([[  -2.5234,    2.1833,  356.9958,    4.7638, -341.5815,  -41.9627],
        [  -2.5323,    2.2540,  350.5453,    4.5372, -346.2897,  -39.3455],
        [  -2.5441,    2.3328,  356.5858,    5.0860, -363.8193,  -44.8737],
        [  -2.2138,    1.4416,  328.3057,    2.8818, -232.8502,  -21.7091],
        [  -2.3361,    2.1650,  375.0593,    5.2776, -391.4387,  -42.8116],
        [  -2.4083,    1.8468,  344.6872,    4.3046, -289.9984,  -36.4636],
        [  -2.3557,    2.4768,  381.0054,    5.7766, -459.4767,  -46.4667],
        [  -2.5538,    1.8628,  365.7951,    3.6706, -289.4641,  -33.1539],
        [  -2.2799,    1.1058,  341.1504,    2.1297, -175.5862,  -17.2102]],
       grad_fn=<IndexBackward0>)


In [114]:
# calculate std/mean to see the variation of the signature
std_over_mean = abs(std_signature/mean_signature)
print("std/mean ", std_over_mean)

std/mean  tensor([0.0527, 0.2281, 0.0469, 0.2763, 0.2649, 0.2858],
       grad_fn=<AbsBackward0>)
